In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile 
import sqlite3
%matplotlib inline

In [2]:
with open('../../data/rt.movie_info.tsv.gz') as f:
    print(f)
with open('../../data/rt.reviews.tsv.gz') as f:
    print(f)
with open('../../data/im.db.zip') as f:
    print(f)
with open('../../data/tmdb.movies.csv.gz') as f:
    print(f)
with open('../../data/tn.movie_budgets.csv.gz') as f:
    print(f)

<_io.TextIOWrapper name='../../data/rt.movie_info.tsv.gz' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='../../data/rt.reviews.tsv.gz' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='../../data/im.db.zip' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='../../data/tmdb.movies.csv.gz' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='../../data/tn.movie_budgets.csv.gz' mode='r' encoding='cp1252'>


In [3]:
#Extract imdb sql DB
with zipfile.ZipFile('../../data/im.db.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path='data/')

In [4]:
#Connect to IMDB DB and get schema
con = sqlite3.connect('../../data/im.db')
pd.read_sql("""

SELECT *
FROM sqlite_master

""",con)

type           name       tbl_name  rootpage  \
0  table   movie_basics   movie_basics         2   
1  table      directors      directors         3   
2  table      known_for      known_for         4   
3  table     movie_akas     movie_akas         5   
4  table  movie_ratings  movie_ratings         6   
5  table        persons        persons         7   
6  table     principals     principals         8   
7  table        writers        writers         9   

                                                 sql  
0  CREATE TABLE "movie_basics" (\n"movie_id" TEXT...  
1  CREATE TABLE "directors" (\n"movie_id" TEXT,\n...  
2  CREATE TABLE "known_for" (\n"person_id" TEXT,\...  
3  CREATE TABLE "movie_akas" (\n"movie_id" TEXT,\...  
4  CREATE TABLE "movie_ratings" (\n"movie_id" TEX...  
5  CREATE TABLE "persons" (\n"person_id" TEXT,\n ...  
6  CREATE TABLE "principals" (\n"movie_id" TEXT,\...  
7  CREATE TABLE "writers" (\n"movie_id" TEXT,\n  ...

In [5]:
#Load 'The Movie DB' dataset
tmdb_df = pd.read_csv('../../data/tmdb.movies.csv.gz',index_col=0)
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [6]:
tmdb_df

genre_ids      id original_language  \
0          [12, 14, 10751]   12444                en   
1      [14, 12, 16, 10751]   10191                en   
2            [12, 28, 878]   10138                en   
3          [16, 35, 10751]     862                en   
4            [28, 878, 12]   27205                en   
...                    ...     ...               ...   
26512             [27, 18]  488143                en   
26513             [18, 53]  485975                en   
26514         [14, 28, 12]  381231                en   
26515      [10751, 12, 28]  366854                en   
26516             [53, 27]  309885                en   

                                     original_title  popularity release_date  \
0      Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                          How to Train Your Dragon      28.734   2010-03-26   
2                                        Iron Man 2      28.515   2010-05-07   
3                                         Toy Story      28.005   1995-11-22   
4                                         Inception      27.920   2010-07-16   
...                                             ...         ...          ...   
26512                         Laboratory Conditions       0.600   2018-10-13   
26513                               _EXHIBIT_84xxx_       0.600   2018-05-01   
26514                                  The Last One       0.600   2018-10-01   
26515                                  Trailer Made       0.600   2018-06-22   
26516                                    The Church       0.600   2018-10-05   

                                              title  vote_average  vote_count  
0      Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
1                          How to Train Your Dragon           7.7        7610  
2                                        Iron Man 2           6.8       12368  
3                                         Toy Story           7.9       10174  
4                                         Inception           8.3       22186  
...                                             ...           ...         ...  
26512                         Laboratory Conditions           0.0           1  
26513                               _EXHIBIT_84xxx_           0.0           1  
26514                                  The Last One           0.0           1  
26515                                  Trailer Made           0.0           1  
26516                                    The Church           0.0           1  

[26517 rows x 9 columns]

In [7]:
#Load review info from Rotten Tomatoes
rt_review_df = pd.read_csv('../../data/rt.reviews.tsv.gz',delimiter='\t',encoding = 'unicode_escape')
rt_review_df['publisher'].fillna('N/A',inplace = True)

# Don't need written review. Date in this instance is review date and irrelevant
rt_review_df.drop(columns = ['review','date'], inplace = True)
# rt_review_df = rt_review_df[rt_review_df['year'] >=2012]

rt_review_df
rt_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   rating      40915 non-null  object
 2   fresh       54432 non-null  object
 3   critic      51710 non-null  object
 4   top_critic  54432 non-null  int64 
 5   publisher   54432 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


In [8]:
top10_publishers_df = rt_review_df[rt_review_df['publisher'].isin(rt_review_df['publisher'].value_counts()[:10].index)]

In [9]:
#Load movie info from Rotten Tomatoes
rt_movie_df = pd.read_csv('../../data/rt.movie_info.tsv.gz',delimiter = '\t')

# NOTE: No movie title provided in dataset. And unique identifier 'id' cannot be matched
# to another db.

#Create list of genres for each record
rt_movie_df['genre list'] = rt_movie_df['genre'].str.split('|')

#'currency' and 'box_office' columns are about 4/5 empty. Dropping these columns. 
# Other datasets are morecomplete and this data can be gathered from there instead 
# Dropping synopsis and studio as well
# rt_movie_df.drop(columns = ['synopsis','currency','box_office','studio'],inplace=True)

# Converting theater date to datetime object and creating year column
rt_movie_df['theater_date'] = pd.to_datetime(rt_movie_df['theater_date'])
rt_movie_df['year'] = rt_movie_df['theater_date'].dt.year

# Dropping records prior to 2012 (last 10yrs)
# rt_movie_df = rt_movie_df[rt_movie_df['year'] >=2012]

rt_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1560 non-null   int64         
 1   synopsis      1498 non-null   object        
 2   rating        1557 non-null   object        
 3   genre         1552 non-null   object        
 4   director      1361 non-null   object        
 5   writer        1111 non-null   object        
 6   theater_date  1201 non-null   datetime64[ns]
 7   dvd_date      1201 non-null   object        
 8   currency      340 non-null    object        
 9   box_office    340 non-null    object        
 10  runtime       1530 non-null   object        
 11  studio        494 non-null    object        
 12  genre list    1552 non-null   object        
 13  year          1201 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(11)
memory usage: 170.8+ KB
